# 4. Intro to Exploratory Data Analysis

In the last chapter, we learned about loading data into R and practiced selecting and summarizing columns and rows of the data. In this chapter, we will learn how to conduct more exploratory analysis focusing on the univariate and bivariate sample distributions. The first half focuses on using base R to create basic plots and summaries. In the second half, we show how to utilize create summary plots using the `GGally` package and tables using the `gt` and `gtsummary` packages.

In [1]:
library(RforHDSdata) 
suppressPackageStartupMessages(library(GGally)) 
library(gt) 
library(gtsummary)

ERROR: Error in library(gt): there is no package called ‘gt’


## Univariate Distributions

We will use a samples of the National Health and Nutrition Examination Survey ([NHANES](https://www.cdc.gov/nchs/nhanes/index.htm)). The sample contains lead, blood pressure, BMI, smoking status, alcohol use, and demographic variables from NHANES 1999-2018. There are 31,625 observations in this sample. Use the help operator `?NHANESsample` to read the variable descriptions.

In [ ]:
data(NHANESsample)
dim(NHANESsample)
names(NHANESsample)

To start our exploration, we will look at whether there are any missing values. We use the `complete.cases()` function to observe that there are no complete cases. We also see that the subsequent blood pressure measurements and alcohol use have the highest percentage of missing values. For demonstration, we chose to only keep the first systolic an diastolic blood pressure measurement and do a complete case analysis using the `na.omit()` function to define our complete data frame `nhanes_df`. 

In [ ]:
sum(complete.cases(NHANESsample))
apply(NHANESsample, 2, function(x) sum(is.na(x)))/nrow(NHANESsample)

In [ ]:
nhanes_df <- na.omit(subset(NHANESsample, select= -c(SBP2, SBP3, SBP4, DBP2, DBP3, DBP4)))

In the last chapter, we introduced the `table()` and `summary()` functions to quickly summarize categorical and quantitative vectors. 

In [ ]:
table(nhanes_df$SMOKE)
summary(nhanes_df$YEAR)

We decide to select the most recent observations from NHANES 2017-2018 for our analysis in this chapter. We use the `subset()` function to select these rows. 

In [ ]:
nhanes_df <- subset(nhanes_df, nhanes_df$YEAR==2017)

As shown above, smoking status has been coded into three categories. We want to create a new variable to represent whether someone has ever smoked. To do so, we use the `ifelse()` function. This function allows us to create a new vector using some logic. The logic captured by this function is that we will take one value if we meet some condition(s) and we will use a second value otherwise. The first argument is a vector of TRUE/FALSE values representing the conditions, the next argument is the value to use if we meet the condition(s), and the last argument is the value to use otherwise. We use this function to create a new vector `EVER_SMOKE` that equals "Yes" for those who are either still smoking or quit smoking and "No" otherwise. 

In [ ]:
nhanes_df$EVER_SMOKE <- ifelse(nhanes_df$SMOKE %in% c("QuitSmoke", "StillSmoke"), "Yes", "No")
table(nhanes_df$EVER_SMOKE)

The `summary()` and `table()` functions allow us to summarize the univariate sample distributions of columns. We may also want to plot these distributions. We saw in the last chapter that the `hist()` function creates a histogram plot. Below we use this to plot a histogram of the log transformation of the lead column.

In [ ]:
hist(log(nhanes_df$LEAD))

If we want to polish this figure, we can use some of the other optional arguments to the function.  For example, we may want to update the text `log(nhanes_df$lead)` in the title and x-axis. Below we update the [color](http://www.stat.columbia.edu/~tzheng/files/Rcolor.pdf), labels, and number of bins for the plot. The argument `breaks` specifies the number of bins to use to create the histogram, `col` specifies the color, `main` specified the title of the plot, and `xlab` specifies the x-axis label (using `ylab` would specify the y-axis label). Read the documentation `?hist` for the full list of arguments available.

In [ ]:
hist(log(nhanes_df$LEAD), breaks = 30, col="blue", main="Histogram of Log Blood Lead Level",
    xlab="Log Blood Lead Level")

For categorical variables, we may want to plot the counts in each category using a bar plot. The function `barplot()` asks us to specify the `names` and `heights` of the bars. To do so, we will need to store the counts for each category. Again, we update the color and labels.

In [ ]:
smoke_counts <- table(nhanes_df$SMOKE)
barplot(height=smoke_counts, names=names(smoke_counts), col="violetred",
       xlab="Smoking Status", ylab="Frequency")

With a bar plot, we can even specify a different color for each bar. To do so, `col` must be a vector of specified colors with the same length as the number of categories.

In [ ]:
barplot(height=smoke_counts, names=names(smoke_counts), col=c("orange","violetred","blue"),
       xlab="Smoking Status", ylab="Frequency")

### Practice Question

Create a barplot of the percentage of values in each `LEAD_QUANTILE` category.

In [ ]:
# Solution:

## Bivariate Distributions

We now turn our attention to the relationship between multiple columns. When we have two categorical variables, we can use the `table()` function to find the counts across all combinations. Below, we look at the distribution in smoking status by sex. We observe that a higer percentage of female participants have never smoked.

In [ ]:
table(nhanes_df$SMOKE, nhanes_df$SEX)

To look at the sample distribution of a continuous variable stratified by a cateogrical variable, we could call the `summary()` function for each subset of the data. Below we look at the distribution of blood lead level by sex. 

In [ ]:
summary(nhanes_df$LEAD[nhanes_df$SEX=="Female"])
summary(nhanes_df$LEAD[nhanes_df$SEX=="Male"])

We could also observe this visually through a box plot. When given one categorical variable and one continuous variable, the `plot()` function creates a box plot. By default, the first argument is the x-axis variable and second argument is the y-axis variable. 

In [ ]:
plot(nhanes_df$SEX, log(nhanes_df$LEAD), ylab="Log Blood Lead Level", xlab="Sex")

Alternatively, we could use the `boxplot()` which can be passed a formula. A formula is a string representation of how to group the data where the left hand side is the continuous variable and the right hand side is one or more categorical variables to group by. In the case below, we group by multiple columns, `SEX` and `EVER_SMOKE`, so our formula is `log(LEAD)~SEX+EVER_SMOKE`. The second argument to the function specifies the data.  We specify the column colors to show the link between the box plots shown. 

In [ ]:
boxplot(log(LEAD)~SEX+EVER_SMOKE, data=nhanes_df, col=c("orange", "blue", "orange", "blue"),
       xlab="Sex : Ever Smoked", ylab = "Log Blood Lead Level")

To visualize the bivariate distribution between two continuous variables, we can use a scatter plot. To create a scatter plot, we use the `plot()` function again. We use this function to show the relationship between systolic and diastolic blood pressure. 

In [ ]:
plot(nhanes_df$SBP1, nhanes_df$DBP1, col="blue", xlab="Systolic Blood Pressure",
    ylab="Diastolic Blood Pressure")

The two measures of blood pressure look highly correlated. We can calculate their Pearson and Spearman correlation using the `cor()` function. The default method is the Pearson correlation, but we can also specify to calculate the Kendall or Spearman correlation by specifying the method. 

In [ ]:
cor(nhanes_df$SBP1, nhanes_df$DBP1)
cor(nhanes_df$SBP1, nhanes_df$DBP1, method="spearman")

We may also want to add some extra information to our plot above. This time instead of specifying the color manually, we use the column `hyp`, an indicator for hypertension, to specify the color. We have to make sure this vector is a factor for R to color by group. Additionally, we add a blue vertical and horizontal line using the `abline()` function to mark cutoffs for hypertension. Even though this function is called after `plot()`, the lines are automatically added to the current plot.

In [ ]:
plot(nhanes_df$SBP1, nhanes_df$DBP1, col=as.factor(nhanes_df$HYP), 
     xlab="Systolic Blood Pressure",
     ylab="Diastolic Blood Pressure")
abline(v=130, col="blue")
abline(h=80, col="blue")

The plots above are all displayed as a single figure. If we want to combine plots, we can specify the graphical parameters using the `par()` function where we update the argument `mfrow=c(nrow, ncol)` with the number of columns and rows we would like to use for our figures. We use this to the distribution of log blood lead level between those with and without hypertension next to the plot above. 

In [ ]:
par(mfrow=c(1,2))

# boxplot
boxplot(log(LEAD)~HYP, data=nhanes_df, xlab="Hypertension", ylab="Log Blood Lead Level")

# scatterplot
plot(nhanes_df$SBP1, nhanes_df$DBP1, col=as.factor(nhanes_df$HYP), 
     xlab="Systolic Blood Pressure",
     ylab="Diastolic Blood Pressure")
abline(v=130, col="blue")
abline(h=80, col="blue")

We then reset to only display a single plot for future images using the `par()` function again.

In [ ]:
par(mfrow=c(1,1))

### Practice Question

Create three boxplots (one for each education level) of income by BMI category and arrange them next to each other using the par() function.

In [ ]:
# Solution:

## Autogenerated Plots

Above, we learned some new functions for visualizing the relationship between variables. The `GGally` package contains some useful functions for looking at multiple univariate and bivariate relationships at the same time. The `ggpairs()` function takes the data as its first argument. By default, it will plot the pairwise distributions for all columns, but we can also specify to only select a subset of columns using the `columns` argument. You can see below that it plots the box plots and density plots for each univariate sample distribution. It then plots the bivariate distributions and calculates the Pearson correlation for all pairs of continuous variables. That's a lot of information!

In [ ]:
ggpairs(nhanes_df, columns = c("SEX", "AGE", "LEAD", "SBP1", "DBP1"))

Another useful function in this package is the `ggcorr()` function. In this function, it expects us to pass in a data frame with only numeric columns and it displays the correlation between all pairs where the color of each grid cell indicates the strength of the correlation. The additional argument `label=TRUE` specifies to put the correlation value on each grid cell. This is a useful way to identify pairs of strongly correlated variables. 

In [ ]:
nhanes_cont <- nhanes_df[,c("AGE", "LEAD", "SBP1", "DBP1")]
ggcorr(nhanes_cont, label=TRUE)

## Tables

Another useful way to display information about your data is through tables. For example, it is standard practice in articles to have the first table in the paper give information about the study sample such as the mean and standard deviation for all continuous variables and the proportions for categorical variables. The `gt` package is designed to create beautiful tables that can include footnotes, titles, column labels, etc. The `gtsummary` package is an extension of this package that can create summary tables. We will focus on the latter but come back to creating nice tables in Chapter 12.

To start, we create a gt object using the first six rows of our data using the `gt()` function. A gt object is a special type of table and in R Markdown this would be be displayed that way. Since we are using this within a Jupyter notebook, we need to pass this to another function to make sure it displays correctly as HTML. The pipe operator `%>%` passes the result on the left hand side as the first argument to the function on the right hand side. You can see the difference in the formatting as opposed to printing the data.

In [ ]:
gt(head(nhanes_df)) %>% gt:::as.tags.gt_tbl()

We now show how to use the `tbl_summary()` function in the `gtsummary` package. The first argument to this function is the data frame again. By default, this function will summarize all the variables in the data. Instead, we use the `include` argument to specify a list of variables to include. We then pipe this result to `as_gt()` which creates a gt table from the summary output before passing to the last function again to display the HTML table. Note that the table computes the total number of observations and computes the proportions for categorical variables and the interquartile range for continuous variables.

In [ ]:
tbl_summary(nhanes_df, include= c("SEX", "RACE", "AGE", "EDUCATION", "SMOKE",
                                  "BMI_CAT", "LEAD", "SBP1", "DBP1", "HYP")) %>% 
  as_gt() %>% 
  gt:::as.tags.gt_tbl()

We can update some of the arguments to update our table. This time we specify that we want to stratify our table by the hypertension status, which summarizes the data by this grouping. Additionally, we change how continuous variables are summarized by specifying that we want to report the mean and standard deviation. We do this using the `statistic` argument. The documentation has help for formatting this argument depending on which statistics you would like to display.

In [ ]:
tbl_summary(nhanes_df, include= c("SEX", "RACE", "AGE", "EDUCATION", "SMOKE",
                                  "BMI_CAT", "LEAD", "SBP1", "DBP1", "HYP"),
           by = "HYP", statistic = list(all_continuous() ~ "{mean} ({sd})")) %>% 
  as_gt() %>% 
  gt:::as.tags.gt_tbl()

## Recap Video

## Exercises

For these exercises, we will be using the **NHANES** data.

1. Using both numerical and graphical summaries, describe the distribution of the first diastolic blood pressure reading `DBP1` and the age `AGE` among study participants, respectively. Then, create a column called `INCOME_CAT` with two categories: low income for those whose income is at most 2 and normal otherwise.

In [ ]:
## solutions:
# dbp
summary(nhanes_df$DBP1)
hist(nhanes_df$DBP1,breaks = 30, col="pink", main="Histogram of Diastolic Blood Pressure",
    xlab="Diastolic Blood Pressure")

# age
summary(nhanes_df$AGE)
hist(nhanes_df$AGE, breaks = 30, col="yellow", main="Histogram of Age",
    xlab="Age")

# income
nhanes_df$INCOME_CAT <- ifelse(nhanes_df$INCOME<=2,"Low","Normal")

2. Subset the data to adults between the ages of 20 and 55. Then, explore how blood pressure varies by age and gender among this age group. Is there a visible trend in blood pressure with increasing age among either sex? 

In [ ]:
nhanes_df.subset.female <- subset(nhanes_df, 
                              nhanes_df$AGE>=20 & nhanes_df$AGE<=55 & SEX == "Female")
nhanes_df.subset.male <- subset(nhanes_df, 
                              nhanes_df$AGE>=20 & nhanes_df$AGE<=55 & SEX == "Male")
plot(nhanes_df.subset.female$AGE, nhanes_df.subset.female$DBP1, xlab = "age", ylab = "Diastolic Blood Pressure", col = "red")
plot(nhanes_df.subset.male$AGE, nhanes_df.subset.male$DBP1, xlab = "age", ylab = "Diastolic Blood Pressure", col = "blue")

4. For males between the ages of 50-59, compare blood pressure across race as reported in the race variable. Then, create a summary table stratified by the `RACE` categories and report the mean, standard deviation, minimum and maximum values for all continuous variables.

In [ ]:
subset.males <- subset(nhanes_df, 
                       nhanes_df$SEX=="Male" & nhanes_df$AGE>=50 & nhanes_df$AGE<=59)
subset.males$RACE <- as.factor(subset.males$RACE)
boxplot(subset.males$DBP1 ~ subset.males$RACE, 
        xlab = "Race", ylab = "Diastolic Blood Pressure", col = "pink")

tbl_summary(subset.males, include= c("SEX", "RACE", "AGE", "EDUCATION", "SMOKE",
                                  "BMI_CAT", "LEAD", "SBP1", "DBP1", "HYP"),
           by = "RACE", statistic = list(all_continuous() ~ "{mean} ({sd}, {min}, {max})")) %>% 
  as_gt() %>% 
  gt:::as.tags.gt_tbl()